In [619]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import EngFormatter
import scipy.signal as sp
%matplotlib qt5

In [620]:
formatter0 = EngFormatter(unit='Hz')
rc('font',family='serif')
plt.rcParams['figure.figsize'] = (6,3)
plt.rcParams['figure.dpi'] = 150

In [621]:
file = open("data.bin", "r")
interleaved_data = np.fromfile(file, np.uint8)
file.close()

In [622]:
I_data_raw = interleaved_data[0:len(interleaved_data):2]
Q_data_raw = interleaved_data[1:len(interleaved_data):2]

I_samples = (I_data_raw-127.5)/127.5
Q_samples = (Q_data_raw-127.5)/127.5

complex_data = I_samples + 1j*Q_samples

In [623]:
plt.figure(1)
plt.plot(np.abs(complex_data))
plt.xlabel("Time Bins")
plt.ylabel("Normalized Amplitude")
plt.xlim(0,len(I_samples))
#plt.xaxis.set_major_formatter(formatter0)
plt.title("In-Phase Data (5 Bursts: OOK, 4-ASK, DBPSK, DQPSK, D8PSK)")
plt.grid()
plt.show()

In [605]:
start = 68027
end = 93500

In [606]:
DATA_I = I_samples[start:end]
DATA_Q = Q_samples[start:end]
DATA_mag = np.abs(complex_data)[start:end]

In [607]:
DATA_I = DATA_I / DATA_mag.max()
DATA_Q = DATA_Q / DATA_mag.max()

DATA_signal = DATA_I + 1j*DATA_Q

In [608]:
#Scatter Plot of the DBPSK data.
f1, ax1 = plt.subplots(1,1)
ax1.scatter(np.real(DATA_signal), np.imag(DATA_signal),facecolors='none', edgecolors='r') 
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.grid()
plt.show()

In [609]:
#create a time array
fs = 2.4E6 #Carrier frequency of SDR
dt = 1/(fs) #Timestep between samples 
freq = np.fft.fftfreq(len(DATA_signal),dt)
bins=np.arange(0,len(DATA_signal),1)
t=bins*dt

In [610]:
f2, ax2 = plt.subplots(1,1)
ax2.plot(bins,np.abs(DATA_signal),'r')
plt.xlabel("bins") 
plt.ylabel("Normalized Magnitude")
plt.grid()
plt.show()

In [611]:
#FFT before corse frequency correction 
DATA_SIGNAL_O = np.fft.fft(DATA_signal)
DATA_MAG_O = 10*np.log10(np.abs(DATA_SIGNAL_O))

In [612]:
Δ_Φ = np.diff(np.unwrap((np.angle(DATA_signal[2:227]))))
Δ_f = np.median(Δ_Φ)  / (np.pi*2*dt)  
print(" Corse Frequency Offset:",Δ_f,'Hz')

 Corse Frequency Offset: 653030.8226679405 Hz


In [613]:
#FFT after corse frequency correction
DATA_signal = DATA_signal*(np.cos(2*np.pi*(-1*Δ_f)*t) + 1j*np.sin(2*np.pi*(-1*Δ_f)*t))
DATA_SIGNAL = np.fft.fft(DATA_signal)
DATA_MAG = 10*np.log10(np.abs(DATA_SIGNAL))

In [614]:
f3, ax3 = plt.subplots(1,1)
ax3.xaxis.set_major_formatter(formatter0)
ax3.plot(freq,DATA_MAG_O,'r',label='Before Δf correction')
ax3.plot(freq,DATA_MAG,'b',label='After Δf correction')
plt.xlabel("frequency (Hz)") 
plt.ylabel("Magnitude (dB)")
plt.legend(loc='upper right')
plt.ylim(0,DATA_MAG_O.max())
plt.grid()
plt.show()

In [615]:
#DFT of |DBPSK_signal|
DATA_SIGNAL_ABS = np.fft.fft(np.abs(DATA_signal))
DATA_MAG_ABS = 10*np.log10(np.abs(DATA_SIGNAL_ABS))
DATA_SIGNAL_Φ = np.angle(DATA_SIGNAL_ABS)

In [616]:
f4, ax4 = plt.subplots(1,1)
ax4.xaxis.set_major_formatter(formatter0)
ax4.plot(freq,DATA_MAG_ABS,'r')
plt.xlabel("frequency (Hz)") 
plt.ylabel("Magnitude (dB)")
plt.xlim(0,freq.max())
plt.grid()
plt.show()

In [617]:
#Thus we obtain the frequency of the clock by lDBPSKing for the largest spike above the noise.
Index_max = 800+(DATA_MAG_ABS[800:1200].argmax())
f_clk=freq[Index_max]
freq_δ = np.abs(freq[0]-freq[1])/2
#Now need to obtain the phase of the clock.
# I did this by using the index function which searches the array and returns the bin where that value is located
for i, j in enumerate(freq):
    if (f_clk-freq_δ) < j < (f_clk+freq_δ):
            freq_bin=(i)
f_clk = freq[freq_bin]
Φ =DATA_SIGNAL_Φ[freq_bin]
print('Clock Frequency:',f_clk,'Hz')
print('Φ:',Φ,'radians')
print('bin:',freq_bin)

Φ = DATA_SIGNAL_Φ[freq_bin]
f = freq[freq_bin]
ω = 2 * np.pi * f
NCO = NCO = np.cos((ω*t)+Φ) + 1j*np.sin((ω*t)+Φ)
peak_bins = sp.find_peaks(np.real(NCO))
ϕ_out = 0
freq_out = 0
bandwidth = 0.002
β = np.sqrt(bandwidth)
length = len(peak_bins[0])
symbol_data = []
error = []
for k in peak_bins[0]:
        symbol_data.append(DATA_signal[k])
sym_length = len(symbol_data) - 2
i=0
while  i < length:
    ΔΦ = np.angle(symbol_data[i]*np.conj(NCO[peak_bins[0][i]]))
    error.append(ΔΦ)
    freq_out += bandwidth * ΔΦ
    ϕ_out += β * ΔΦ * freq_out
    NCO = NCO * np.e**(-1j*ϕ_out)
    peak_bins = sp.find_peaks(np.real(NCO))
    length = len(peak_bins[0])
    if sym_length < i:
        symbol_data.append(DATA_signal[peak_bins[0][i]])
    else:
        symbol_data[i] = (DATA_signal[peak_bins[0][i]])
    i +=1
plt.figure(6)
plt.plot(np.unwrap(error))
plt.grid()
plt.show()

Clock Frequency: 94971.14591920857 Hz
Φ: -2.8986374289822967 radians
bin: 1008


In [618]:
plt.figure(7)
plt.plot(bins,np.abs(DATA_signal))
plt.plot(bins,np.real(NCO))
plt.xlim(0,290)
plt.grid()
plt.show()

sym_max = np.abs(symbol_data).max()
norm_data = np.real(symbol_data)/sym_max + 1j*np.imag(symbol_data)/sym_max
        
plt.figure(8)
plt.scatter(np.real(norm_data),np.imag(norm_data) ,linewidths=0.1,facecolors='b', edgecolors='b')
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("OOK: Scatter plot, downsampled")
plt.grid()
plt.show()

In [481]:
peak_bins = sp.find_peaks(np.real(NCO))

In [482]:
symbol_data = []
ΔΦ = 0
# First Symbol
b = peak_bins[0][0]
symbol_data.append(DATA_signal[b])
# First Symbol Prediction
p0 = 1 + 1j*0
# Phase difference
ΔΦ += np.angle(symbol_data[0]*np.conj(p0))
symbol_data[0] = symbol_data[0] * np.e**(-1j*ΔΦ)
#Second Symbol
b = peak_bins[0][1]
symbol_data.append(DATA_signal[b])
# Second Symbol Prediction
p1 = p0
# Phase difference
print(np.angle(symbol_data[1]*np.conj(p1)))
ΔΦ += np.angle(symbol_data[1]*np.conj(symbol_data[0]))
symbol_data[1] = symbol_data[1] * np.e**(-1j*ΔΦ)

-0.6400889362868709


In [483]:
sym_max = np.abs(symbol_data).max()
norm_data = np.real(symbol_data)/sym_max + 1j*np.imag(symbol_data)/sym_max
plt.figure(8)
plt.scatter(np.real(norm_data),np.imag(norm_data) ,linewidths=0.1,facecolors='b', edgecolors='b')
#plt.scatter(np.real(p1),np.imag(p1) ,linewidths=0.1,facecolors='b', edgecolors='b')
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("OOK: Scatter plot, downsampled")
plt.xlim(-1.1,1.1)
plt.ylim(-1.1,1.1)
plt.grid()
plt.show()

In [484]:

peak_bins = sp.find_peaks(np.real(NCO))
for k in peak_bins[0]:
        symbol_data.append(DATA_signal[k])

In [485]:

                 
plt.figure(7)
plt.plot(np.unwrap(Error))
plt.grid()
plt.show()

plt.figure(8)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1,facecolors='b', edgecolors='b')
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("DBPSK: Scatter plot, downsampled")
plt.grid()
plt.show()